In [71]:
import os 
list=os.listdir("FFHQ")
list=list[0:100]

In [74]:
import cv2
import time
leye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')
reye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_righteye_2splits.xml')
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
file=open("coordinates_100_Outputsamples.csv","w")
file.write("file,left_eye,right_eye,mouth\n")
for i in list:
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    image = cv2.imread("FFHQ/"+i)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    row=""
    row=row+i+','
    faces=sorted(faces,key=lambda row: row[2]*row[3],reverse=True)
    for (x, y, w, h) in faces[0:1]:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = image[y:y + h, x:x + w]
        roi_lower_gray = gray[y+h//2:y+h,x:x+w]
        roi_lower_color = image[y+h//2:y+h,x:x+w]
        roi_upper_gray = gray[y:y+h*2//3,x:x+w]
        roi_upper_color = image[y:y+h*2//3,x:x+w]
        roi_upper_left_gray = gray[y:y+h*2//3,x+w//2:x+w]
        roi_upper_left_color = image[y:y+h*2//3,x+w//2:x+w]
        roi_upper_right_gray = gray[y:y+h*2//3,x:x+w//2]
        roi_upper_right_color = image[y:y+h*2//3,x:x+w//2]
        reyes = reye_cascade.detectMultiScale(roi_upper_right_gray, scaleFactor=1.1, minNeighbors=5)
        reyes=sorted(reyes,key=lambda row: row[2]*row[3],reverse=True)
        leyes = leye_cascade.detectMultiScale(roi_upper_left_gray, scaleFactor=1.1, minNeighbors=5)
        leyes=sorted(leyes,key=lambda row: row[2]*row[3],reverse=True)
        eyes = eye_cascade.detectMultiScale(roi_upper_gray, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))
        eyes=sorted(eyes,key=lambda row: row[2]*row[3],reverse=True)
        if(eyes.__len__()>=2):
            eyes=eyes[0:2]
        eyes=sorted(eyes,key=lambda row:row[0],reverse=True)
        if (leyes.__len__()==0 or reyes.__len__==0):
            for (ex, ey, ew, eh) in eyes[0:min(2,eyes.__len__())]:
                eye_x = x + ex + ew // 2
                eye_y = y + ey + eh // 2
                cv2.rectangle(image, (x+ex, y+ey),(x+ex+ew, y+ey+eh),(0, 0, 0),2)
                row=row+'"'+str((x+ex, y+ey))+str((x+ex+ew, y+ey+eh))+'"'+","
        else:
            for (ex, ey, ew, eh) in leyes[0:1]:
                eye_x = x + ex + ew // 2
                eye_y = y + ey + eh // 2
                cv2.rectangle(image, (x+ex+w//2, y+ey),(x+ex+ew+w//2, y+ey+eh),(0, 0, 0),2)
                row=row+'"'+str((x+ex+w//2, y+ey))+str((x+ex+ew+w//2, y+ey+eh))+'"'+","
            for (ex, ey, ew, eh) in reyes[0:1]:
                eye_x = x + ex + ew // 2
                eye_y = y + ey + eh // 2
                cv2.rectangle(image, (x+ex, y+ey),(x+ex+ew, y+ey+eh),(0, 0, 0),2)
                row=row+'"'+str((x+ex, y+ey))+str((x+ex+ew, y+ey+eh))+'"'+","
        mouths = mouth_cascade.detectMultiScale(roi_lower_gray, scaleFactor=1.1, minNeighbors=15, minSize=(20, 20))
        mouths=sorted(mouths,key=lambda row: row[2]*row[3],reverse=True)
        for (mx, my, mw, mh) in mouths[0:1]:
            mouth_x = x + mx + mw // 2
            mouth_y = y + my + mh // 2
            cv2.rectangle(image, (x+mx, y+my+h//2),(x+mx+mw, y+my+mh+h//2),(0, 0, 0),2)
            row=row+'"'+str((x+mx, y+my+h//2))+str((x+mx+mw, y+my+mh+h//2))+'"'+","
    # cv2.imshow("Facial Landmarks", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    cv2.imwrite("output/"+i,image)
    row=row+'\n'
    file.write(row)
    
file.close()
